In [1]:
from main import main
from PDconfig import MeLi
from archivosDF import *

main()
ruta_barrios = './DataFrames/MeLi/barrios.csv'
ruta_ciudades = './DataFrames/MeLi/ciudades.csv'
df = abrirDF(MeLi)

JSONDecodeError: Expecting value: line 287 column 3 (char 10847)